In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
# from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_x.map(lambda x: tf.image.crop_to_bounding_box(x, 8, 8, 80, 80))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))
ds_xc = ds_x.map(lambda x: tf.add(x, tf.random.normal(tf.shape(x), 0, 0.2)))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_xcy = tf.data.Dataset.zip((ds_xc, ds_y))
ds_xcy = ds_xcy.shuffle(BATCH_SIZE*2)

In [8]:
ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_trainc = ds_xcy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [9]:
data_aug = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomContrast(0.2)
    # RandomZoom(0.2, 0.2)
])

In [10]:
model = Sequential()

model.add(data_aug)

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [11]:
ds_trainc = ds_trainc.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_trainc,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 158s - loss: 0.4934 - accuracy: 0.8151 - val_loss: 0.6340 - val_accuracy: 0.7718
Epoch 2/8
3095/3095 - 153s - loss: 0.3775 - accuracy: 0.8568 - val_loss: 0.4885 - val_accuracy: 0.8300
Epoch 3/8
3095/3095 - 153s - loss: 0.3252 - accuracy: 0.8821 - val_loss: 0.6837 - val_accuracy: 0.8010
Epoch 4/8
3095/3095 - 153s - loss: 0.3080 - accuracy: 0.8917 - val_loss: 0.6441 - val_accuracy: 0.8258
Epoch 5/8
3095/3095 - 153s - loss: 0.2993 - accuracy: 0.8951 - val_loss: 0.6028 - val_accuracy: 0.8377
Epoch 6/8
3095/3095 - 153s - loss: 0.2932 - accuracy: 0.8996 - val_loss: 1.1142 - val_accuracy: 0.7259
Epoch 7/8
3095/3095 - 153s - loss: 0.2895 - accuracy: 0.9022 - val_loss: 0.4318 - val_accuracy: 0.8620
Epoch 8/8
3095/3095 - 153s - loss: 0.2852 - accuracy: 0.9028 - val_loss: 1.1458 - val_accuracy: 0.7497


In [12]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 153s - loss: 0.2352 - accuracy: 0.9273 - val_loss: 0.4274 - val_accuracy: 0.8629
Epoch 2/8
3095/3095 - 153s - loss: 0.2277 - accuracy: 0.9304 - val_loss: 0.3277 - val_accuracy: 0.8896
Epoch 3/8
3095/3095 - 153s - loss: 0.2250 - accuracy: 0.9321 - val_loss: 0.3011 - val_accuracy: 0.8998
Epoch 4/8
3095/3095 - 153s - loss: 0.2229 - accuracy: 0.9331 - val_loss: 0.2897 - val_accuracy: 0.9091
Epoch 5/8
3095/3095 - 153s - loss: 0.2197 - accuracy: 0.9342 - val_loss: 0.4795 - val_accuracy: 0.8265
Epoch 6/8
3095/3095 - 153s - loss: 0.2178 - accuracy: 0.9359 - val_loss: 0.4013 - val_accuracy: 0.8858
Epoch 7/8
3095/3095 - 153s - loss: 0.2151 - accuracy: 0.9361 - val_loss: 0.2113 - val_accuracy: 0.9399
Epoch 8/8
3095/3095 - 153s - loss: 0.2123 - accuracy: 0.9375 - val_loss: 0.3429 - val_accuracy: 0.8889


In [13]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
1548/1548 - 153s - loss: 0.1744 - accuracy: 0.9516 - val_loss: 0.1661 - val_accuracy: 0.9541
Epoch 2/8
1548/1548 - 151s - loss: 0.1653 - accuracy: 0.9538 - val_loss: 0.3556 - val_accuracy: 0.8987
Epoch 3/8
1548/1548 - 151s - loss: 0.1595 - accuracy: 0.9552 - val_loss: 0.1603 - val_accuracy: 0.9553
Epoch 4/8
1548/1548 - 151s - loss: 0.1560 - accuracy: 0.9563 - val_loss: 0.3224 - val_accuracy: 0.9059
Epoch 5/8
1548/1548 - 151s - loss: 0.1521 - accuracy: 0.9569 - val_loss: 0.1776 - val_accuracy: 0.9472
Epoch 6/8
1548/1548 - 151s - loss: 0.1495 - accuracy: 0.9575 - val_loss: 0.1837 - val_accuracy: 0.9455
Epoch 7/8
1548/1548 - 160s - loss: 0.1483 - accuracy: 0.9576 - val_loss: 0.2463 - val_accuracy: 0.9270
Epoch 8/8
1548/1548 - 159s - loss: 0.1461 - accuracy: 0.9588 - val_loss: 0.1335 - val_accuracy: 0.9623


In [14]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
774/774 - 156s - loss: 0.1347 - accuracy: 0.9625 - val_loss: 0.1287 - val_accuracy: 0.9648
Epoch 2/8
774/774 - 151s - loss: 0.1327 - accuracy: 0.9633 - val_loss: 0.1272 - val_accuracy: 0.9658
Epoch 3/8
774/774 - 151s - loss: 0.1318 - accuracy: 0.9637 - val_loss: 0.1232 - val_accuracy: 0.9675
Epoch 4/8
774/774 - 152s - loss: 0.1306 - accuracy: 0.9637 - val_loss: 0.1240 - val_accuracy: 0.9671
Epoch 5/8
774/774 - 152s - loss: 0.1301 - accuracy: 0.9639 - val_loss: 0.1237 - val_accuracy: 0.9669
Epoch 6/8
774/774 - 154s - loss: 0.1283 - accuracy: 0.9642 - val_loss: 0.1229 - val_accuracy: 0.9666
Epoch 7/8
774/774 - 165s - loss: 0.1281 - accuracy: 0.9643 - val_loss: 0.1221 - val_accuracy: 0.9669
Epoch 8/8
774/774 - 163s - loss: 0.1277 - accuracy: 0.9647 - val_loss: 0.1221 - val_accuracy: 0.9670


In [15]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(64).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 155s - loss: 0.1431 - accuracy: 0.9583 - val_loss: 0.1262 - val_accuracy: 0.9648
Epoch 2/2
3095/3095 - 166s - loss: 0.1421 - accuracy: 0.9586 - val_loss: 0.1278 - val_accuracy: 0.9652


In [16]:
history = model.fit(ds_trainc,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 166s - loss: 0.2398 - accuracy: 0.9157 - val_loss: 1.0377 - val_accuracy: 0.8196
Epoch 2/2
3095/3095 - 166s - loss: 0.2206 - accuracy: 0.9235 - val_loss: 0.8742 - val_accuracy: 0.8360


In [17]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
774/774 - 161s - loss: 0.1437 - accuracy: 0.9572 - val_loss: 0.1576 - val_accuracy: 0.9524
Epoch 2/5
774/774 - 156s - loss: 0.1340 - accuracy: 0.9610 - val_loss: 0.1390 - val_accuracy: 0.9597
Epoch 3/5
774/774 - 155s - loss: 0.1301 - accuracy: 0.9630 - val_loss: 0.1217 - val_accuracy: 0.9665
Epoch 4/5
774/774 - 156s - loss: 0.1273 - accuracy: 0.9640 - val_loss: 0.1219 - val_accuracy: 0.9670
Epoch 5/5
774/774 - 153s - loss: 0.1260 - accuracy: 0.9640 - val_loss: 0.1190 - val_accuracy: 0.9675


In [18]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
387/387 - 164s - loss: 0.1325 - accuracy: 0.9623 - val_loss: 0.1986 - val_accuracy: 0.9399
Epoch 2/2
387/387 - 168s - loss: 0.1306 - accuracy: 0.9621 - val_loss: 0.1966 - val_accuracy: 0.9392


In [19]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 165s - loss: 0.1220 - accuracy: 0.9656 - val_loss: 0.1185 - val_accuracy: 0.9687
Epoch 2/10
387/387 - 161s - loss: 0.1187 - accuracy: 0.9665 - val_loss: 0.1212 - val_accuracy: 0.9675
Epoch 3/10
387/387 - 161s - loss: 0.1181 - accuracy: 0.9670 - val_loss: 0.1173 - val_accuracy: 0.9691
Epoch 4/10
387/387 - 162s - loss: 0.1165 - accuracy: 0.9678 - val_loss: 0.1170 - val_accuracy: 0.9696
Epoch 5/10
387/387 - 160s - loss: 0.1157 - accuracy: 0.9680 - val_loss: 0.1161 - val_accuracy: 0.9698
Epoch 6/10
387/387 - 165s - loss: 0.1154 - accuracy: 0.9678 - val_loss: 0.1197 - val_accuracy: 0.9681
Epoch 7/10
387/387 - 166s - loss: 0.1150 - accuracy: 0.9680 - val_loss: 0.1258 - val_accuracy: 0.9667
Epoch 8/10
387/387 - 166s - loss: 0.1143 - accuracy: 0.9680 - val_loss: 0.1211 - val_accuracy: 0.9679
Epoch 9/10
387/387 - 161s - loss: 0.1145 - accuracy: 0.9678 - val_loss: 0.1135 - val_accuracy: 0.9700
Epoch 10/10
387/387 - 167s - loss: 0.1132 - accuracy: 0.9685 - val_loss: 0.1163 - 

In [20]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 166s - loss: 0.1125 - accuracy: 0.9689 - val_loss: 0.1116 - val_accuracy: 0.9707
Epoch 2/10
387/387 - 162s - loss: 0.1115 - accuracy: 0.9690 - val_loss: 0.1109 - val_accuracy: 0.9708
Epoch 3/10
387/387 - 169s - loss: 0.1113 - accuracy: 0.9692 - val_loss: 0.1124 - val_accuracy: 0.9700
Epoch 4/10
387/387 - 168s - loss: 0.1110 - accuracy: 0.9694 - val_loss: 0.1124 - val_accuracy: 0.9700
Epoch 5/10
387/387 - 164s - loss: 0.1111 - accuracy: 0.9694 - val_loss: 0.1120 - val_accuracy: 0.9705
Epoch 6/10
387/387 - 164s - loss: 0.1108 - accuracy: 0.9692 - val_loss: 0.1116 - val_accuracy: 0.9709
Epoch 7/10
387/387 - 162s - loss: 0.1113 - accuracy: 0.9693 - val_loss: 0.1112 - val_accuracy: 0.9706
Epoch 8/10
387/387 - 165s - loss: 0.1103 - accuracy: 0.9697 - val_loss: 0.1123 - val_accuracy: 0.9703
Epoch 9/10
387/387 - 164s - loss: 0.1111 - accuracy: 0.9695 - val_loss: 0.1127 - val_accuracy: 0.9704
Epoch 10/10
387/387 - 177s - loss: 0.1105 - accuracy: 0.9696 - val_loss: 0.1126 - 

In [21]:
history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 166s - loss: 0.1103 - accuracy: 0.9696 - val_loss: 0.1119 - val_accuracy: 0.9704
Epoch 2/10
387/387 - 163s - loss: 0.1108 - accuracy: 0.9693 - val_loss: 0.1115 - val_accuracy: 0.9708
Epoch 3/10
387/387 - 171s - loss: 0.1105 - accuracy: 0.9692 - val_loss: 0.1111 - val_accuracy: 0.9712
Epoch 4/10
387/387 - 167s - loss: 0.1107 - accuracy: 0.9696 - val_loss: 0.1109 - val_accuracy: 0.9711
Epoch 5/10
387/387 - 165s - loss: 0.1097 - accuracy: 0.9697 - val_loss: 0.1117 - val_accuracy: 0.9711
Epoch 6/10
387/387 - 167s - loss: 0.1098 - accuracy: 0.9697 - val_loss: 0.1121 - val_accuracy: 0.9708
Epoch 7/10
387/387 - 167s - loss: 0.1094 - accuracy: 0.9698 - val_loss: 0.1114 - val_accuracy: 0.9708
Epoch 8/10
387/387 - 166s - loss: 0.1106 - accuracy: 0.9696 - val_loss: 0.1111 - val_accuracy: 0.9708
Epoch 9/10
387/387 - 177s - loss: 0.1095 - accuracy: 0.9698 - val_loss: 0.1104 - val_accuracy: 0.9713
Epoch 10/10
387/387 - 165s - loss: 0.1091 - accuracy: 0.9701 - val_loss: 0.1110 - 